## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [8]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,Estates at Acqualina\r17901 Collins Ave 1801\r...,2/20/2023,142,"$7,700,000.00",N/A,N/A,Marla Cohen,Acqualina Realty,Yansy Checa,Acqualina Realty,Estates at Acqualina 17901 Collins Ave 1801 Su...,Estates at Acqualina,17901 Collins Ave 1801 Sunny Isles Beach,"(25.9420203, -80.1201787)",25.942020,-80.120179
2,The Fairchild\r3581 E Glencoe St 602\rMiami,2/22/2023,11,"$6,275,000.00",2107,"$2,978.00",Cristian Cardenas,Cardenas Larocca Commercial R.E.,Mauricio Barba PA,Compass Florida LLC,The Fairchild 3581 E Glencoe St 602 Miami,The Fairchild,3581 E Glencoe St 602 Miami,"(25.7401591, -80.21738979999999)",25.740159,-80.217390
3,Turnberry Ocean Club\r18501 Collins Ave 3302\r...,2/21/2023,291,"$6,150,000.00",2180,"$2,821.00",Michael Martirena,Compass Florida LLC,Estela Serrano,"Serrano & Associates Real Estate, LLC",Turnberry Ocean Club 18501 Collins Ave 3302 Su...,Turnberry Ocean Club,18501 Collins Ave 3302 Sunny Isles Beach,"(25.9478144, -80.1201988)",25.947814,-80.120199
4,Grand Bay Tower Condo\r430 Grand Bay Dr 1102\...,2/22/2023,61,"$5,200,000.00",1316,"$3,950.00",Jorge Fernandez,Compass Florida LLC,Brigitte De Langeron,Fortune Christie's Int'l Real Estate,Grand Bay Tower Condo 430 Grand Bay Dr 1102 K...,Grand Bay Tower Condo,430 Grand Bay Dr 1102 Key Biscayne,"(25.693091, -80.157996)",25.693091,-80.157996
5,Grovenor House Condo\r2627 S Bayshore Dr 2703\...,2/21/2023,48,"$3,550,000.00",1345,"$2,640.00",Marilyn Cromer,Douglas Elliman,Ana Sanchez,One Sotheby's Int'l Realty,Grovenor House Condo 2627 S Bayshore Dr 2703 M...,Grovenor House Condo,2627 S Bayshore Dr 2703 Miami,"(25.7322307, -80.2348947)",25.732231,-80.234895
6,Williams Island\r4000 Island Blvd PH1\rAventura,2/21/2023,81,"$2,675,000.00",602,"$4,446.00",Chad Carroll,Douglas Elliman,Perla Bursztein,Canvas Real Estate,Williams Island 4000 Island Blvd PH1 Aventura,Williams Island,4000 Island Blvd PH1 Aventura,"(25.9416063, -80.1348359)",25.941606,-80.134836
7,SLS Lux\r801 S Miami Ave PH5604\rMiami,2/21/2023,69,"$2,500,000.00",990,"$2,524.00",Valka Rodriguez,"Premier Global Brokers, LLC",Jose Caba Jr.,Best International Realty,SLS Lux 801 S Miami Ave PH5604 Miami,SLS Lux,801 S Miami Ave PH5604 Miami,"(25.765952, -80.1930337)",25.765952,-80.193034
8,Trump Palace\r18101 Collins Ave 4606\rSunny Is...,2/24/2023,41,"$2,275,000.00",1215,"$1,873.00",George Dangond,"Stroomline, LLC.",Victoria Romanenko,Golden Keys Inc,Trump Palace 18101 Collins Ave 4606 Sunny Isle...,Trump Palace,18101 Collins Ave 4606 Sunny Isles Beach,"(25.9439355, -80.12004569999999)",25.943935,-80.120046
9,Ocean Two\r19111 Collins Ave 2705\rSunny Isles...,2/23/2023,65,"$2,215,000.00",908,"$2,440.00",Wendy Cohen,One Sotheby's Int'l Realty,Yuliya Kachko,Oceanfront Group Inc,Ocean Two 19111 Collins Ave 2705 Sunny Isles B...,Ocean Two,19111 Collins Ave 2705 Sunny Isles Beach,"(25.9537166, -80.1196968)",25.953717,-80.119697
10,0Murano Grande\r400 Alton Rd 1602\rMiami Beach,2/22/2023,35,"$1,875,000.00",1001,"$1,874.00",Nurdan Yildirim,Florida Realty of Miami Corp,Nurdan Yildirim,Florida Realty of Miami Corp,0Murano Grande 400 Alton Rd 1602 Miami Beach,,0Murano Grande 400 Alton Rd 1602 Miami Beach,"(25.7727159, -80.14041569999999)",25.772716,-80.140416


In [9]:
df.at[10,'building_name']=('Murano Grandeo')
df.at[10,'address_only']=('400 Alton Road 1602 Miami Beach')

## Format Data

In [14]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [15]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=True)
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=True)
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=True)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=True)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [16]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

In [17]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

## HTML Popup Formatter

In [18]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [27]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Listing Broker']
    buyers_agent = row['Buyer Agent']
    psf = row['Sq. Ft.']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [28]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('February 19th - February 25th')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [29]:
m.save('index.html')

## Data snagger

In [30]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [31]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Estates at Acqualina , 17901 Collins Ave 1801 Sunny Isles Beach | Price $7,700,000 | $nan psf | Listing agent: Marla Cohen with Acqualina Realty | Buyer's agent: Yansy Checa with Acqualina Realty | Days on market: 142
Least Expensive
Murano Grandeo, 400 Alton Road 1602 Miami Beach | Price $1,875,000 | $1,874 psf | Listing agent: Nurdan Yildirim with Florida Realty of Miami Corp | Buyer's agent: Nurdan Yildirim with Florida Realty of Miami Corp | Days on market: 35
Highest Price Per Square Foot
Williams Island , 4000 Island Blvd PH1 Aventura | Price $2,675,000 | $4,446 psf | Listing agent: Chad Carroll with Douglas Elliman | Buyer's agent: Perla Bursztein with Canvas Real Estate | Days on market: 81
Lowest Price Per Square Foot
Trump Palace , 18101 Collins Ave 4606 Sunny Isles Beach | Price $2,275,000 | $1,873 psf | Listing agent: George Dangond with Stroomline, LLC. | Buyer's agent: Victoria Romanenko with Golden Keys Inc | Days on market: 41
Most Days on Market
Turnberr

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [32]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [33]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_02272023
